In [1]:
import numpy as np
import sys
from tqdm import tqdm

nDimensions = 3
charge = 2
stepLength = 1.5
nParticles = 2
# step length and square inverse
h = 0.001
h2 = 1000000
alpha = 1
nCycles = 10000
thermalization = 1000

def local_energy(r, alpha):
    r_plus = np.zeros((nParticles, nDimensions))
    r_minus = np.zeros((nParticles, nDimensions))
    r_plus[:] = r
    r_minus[:] = r
    wave_function_minus = 0
    wave_function_plus = 0
    wave_function_current = wave_function(r, alpha)
    
    # Kinetic energy, brute force derivations
    kinetic_energy = 0
    for i in range(nParticles):
        for j in range(nDimensions):
            r_plus[i,j] += h
            r_minus[i,j] -= h
            wave_function_minus = wave_function(r_minus, alpha)
            wave_function_plus = wave_function(r_plus, alpha)
            kinetic_energy -= (wave_function_minus + wave_function_plus - 2 * wave_function_current)
            r_plus[i,j] = r[i,j]
            r_minus[i,j] = r[i,j]
    kinetic_energy = 0.5 * h2 * kinetic_energy / wave_function_current
    
    # Potential energy
    potential_energy = 0
    r_single_particle = 0
    for i in range(nParticles):
        r_single_particle = 0
        for j in range(nDimensions):
            r_single_particle += r[i,j] * r[i,j]
        potential_energy -= charge / np.sqrt(r_single_particle)
    
    # Contribution from electron-electron potential
    r12 = 0
    for i in range(nParticles):
        for j in range(i + 1, nParticles):
            r12 = 0
            for k in range(nDimensions):
                r12 += (r[i,k] - r[j,k]) * (r[i,k] - r[j,k])
            potential_energy += 1 / np.sqrt(r12)
    
    return kinetic_energy + potential_energy

def wave_function(r, alpha):
    argument = 0
    for i in range(nParticles):
        r_single_particle = 0
        for j in range(nDimensions):
            r_single_particle += r[i,j] * r[i,j]
        argument += np.sqrt(r_single_particle)
    return np.exp(-argument * alpha)

def solve_VMC(alpha):
    rOld = np.zeros((nParticles, nDimensions))
    rNew = np.zeros((nParticles, nDimensions))
    waveFunctionOld = 0
    waveFunctionNew = 0
    energySum = 0
    energySquaredSum = 0
    deltaE = 0
    prob = 1.
    # initial trial positions
    rOld = stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)

    rNew = rOld
    # loop over Monte Carlo cycles
    accept = 0
    for cycle in tqdm(range(nCycles), file=sys.stdout, desc='VMC Cycles'):
        waveFunctionOld = wave_function(rOld, alpha)
        direction = (np.random.rand(nParticles, nDimensions)-0.5)
        for i in range(nParticles):
            for j in range(nDimensions):
                rNew[i,j] = rOld[i,j] + stepLength * direction[i,j]
            # Recalculate the value of the wave function
            waveFunctionNew = wave_function(rNew, alpha)
            # Check for step acceptance (if yes, update position, if no, reset position)
            if np.random.rand() <= np.float_power(waveFunctionNew/waveFunctionOld, 2):
                for j in range(nDimensions):
                    rOld[i,j] = rNew[i,j]
                waveFunctionOld = waveFunctionNew
                accept += 1
            else:
                for j in range(nDimensions):
                    rNew[i,j] = rOld[i,j]
            
            ### AVOID division error???????????????????????????????????????????????????????
            if wave_function(rOld, alpha) <= 1E-100:
                tqdm.write("triggered")
                rOld = stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)
                rNew = rOld + stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)

            # update energies
            if cycle > thermalization:
                deltaE = local_energy(rNew, alpha)
                energySum += deltaE
                energySquaredSum += deltaE*deltaE
        if cycle % 20000 == 0: tqdm.write("energy:"+str(round(energySum/((cycle+1) * nParticles)*2*13.6,2))+" (eV) ; acceptance rate = "+str(round(accept/(cycle+1)/nParticles * 100,2))+"; alpha: "+str(alpha))
    energy = energySum/(nCycles * nParticles)*2*13.6
    energySquared = energySquaredSum/(nCycles * nParticles)
    # print(f"Energy: {energy*2*13.6}(eV), Variance: {energySquared-energy**2}")
    return energy


In [2]:
from multiprocessing import Pool

pool = Pool()

alphas = [1.5,1.6,1.7,1.75,1.8,1.9]

energies = list(map(solve_VMC, alphas))

energy:0.0 (eV) ; acceptance rate = 50.0; alpha: 1.5 
VMC Cycles: 100%|██████████| 10000/10000 [00:01<00:00, 6224.31it/s]
energy:0.0 (eV) ; acceptance rate = 50.0; alpha: 1.6 
triggered                                                          
VMC Cycles: 100%|██████████| 10000/10000 [00:01<00:00, 6544.97it/s]
energy:0.0 (eV) ; acceptance rate = 0.0; alpha: 1.7  
VMC Cycles: 100%|██████████| 10000/10000 [00:01<00:00, 6508.77it/s]
energy:0.0 (eV) ; acceptance rate = 50.0; alpha: 1.75
triggered                                                          
VMC Cycles: 100%|██████████| 10000/10000 [00:01<00:00, 6526.14it/s]
energy:0.0 (eV) ; acceptance rate = 50.0; alpha: 1.8 
triggered                                                          
VMC Cycles: 100%|██████████| 10000/10000 [00:01<00:00, 6538.00it/s]
energy:0.0 (eV) ; acceptance rate = 0.0; alpha: 1.9  
triggered                                                          
VMC Cycles: 100%|██████████| 10000/10000 [00:01<00:00, 6623.43it

In [3]:
energies

[-55.053157586315194,
 -64.05964847216886,
 -70.49267379421121,
 -72.75643697863408,
 -74.70834348714828,
 -83.62435767365646]

In [4]:
[(-2*alpha**2+4*charge*alpha-5/4*alpha)*13.6 for alpha in alphas]


[76.5, 77.24799999999999, 77.452, 77.35, 77.112, 76.228]

In [5]:
rOld

NameError: name 'rOld' is not defined